In [ ]:
import os
import pandas as pd
import logging
import time
import json

#리스트합침
def ex(dat,col): 
    result=[]
    for x in dat:
        result.append(x[col])
    return str(result)

# path 지우기
def del_path(str):
    return str.replace("$$[BAT_DATA_HOME]/","").replace("'","")

# 파일에 대한 컬럼정보 추출
def get_columns(blocks,fType):
    cols=['block_id','raw_file_nm','col_id','nm','desc','calc_expression']
    # 각 블럭내 inputs정보 
    df_file = pd.DataFrame(columns=cols)
    for block in data['blocks']:
        if len(block[fType])!=0:
            for inp in block[fType]:
                if (str(type(inp))=="<class 'dict'>") & ('ffd' in inp.keys()):
                    for col in inp['ffd']['columns']:
                        lst= [block['block_id'],del_path(inp['raw_file_nm']),col['col_id'],col['nm'],col['desc'],col['calc_expression']]
                        dd=pd.DataFrame(lst).T
                        dd.columns=cols
                        df_file=df_file.append(dd)

    return df_file


#query 추출
def get_query(qry):
    cols=['block_id','sql']
    df_qry = pd.DataFrame(columns=cols)
    for block in data['blocks']:
        if 'query' in block.keys():
            if len(block['query'])!=0:
                qry=block['query']
                lst=qry.split("\n")
                dd=pd.DataFrame(lst, columns=['sql'])
                dd['block_id']=block['block_id']
                df_qry=df_qry.append(dd[cols])
    return df_qry

#공백제거
def get_strings(st):
    strings=str(st).strip()
    if strings.upper().startswith('SELECT'):
        return strings
    elif strings.upper().startswith('FROM'):
        return strings
    else:
        strings="    "+ strings
        return strings
    
# file 유무 체크 및 path, json data return
def fileCheck(row):
    res=None
    json_path=row['Path']
    json_file=row['FileName']
    js= os.path.join(json_path,json_file)
    if not os.path.isfile(js):
        return 'file notfnd -> ' + js,'er'
    json_data=open(js, encoding='UTF8')
    data = json.load(json_data)
    if data==None: 
        return 'file read error','er'
    return None,js,data

#json -> excel
def procData(json_file,data):
    try:  
        df = pd.DataFrame()
        # for block in data['blocks']:
        #     df=df.append(block,ignore_index=True)
        df=pd.json_normalize(data['blocks'])

        df['input_cnt']=df.inputs.apply(lambda x: len(x))
        df['output_cnt']=df.outputs.apply(lambda x: len(x))

        df['input_files']=df.inputs.apply(lambda x: ex(x,'raw_file_nm'))
        df['output_files']=df.outputs.apply(lambda x: ex(x,'raw_file_nm'))
        df['input_files']=df['input_files'].apply(lambda x: del_path(x))
        df['output_files']=df['output_files'].apply(lambda x: del_path(x))
        if 'outfile1' in df.columns: 
            df['outfile1']=df['outfile1'].apply(lambda x: del_path(str(x))).replace('nan',"")
        # df=df.applymap(lambda x: str(x).replace("$$[BAT_DATA_HOME]/ccr/","").replace("'",""))
        df.drop(columns=['inputs','outputs'],inplace=True)

        #in,out col추출
        df_inFile=h=get_columns(data,"inputs")
        df_outFile=h=get_columns(data,"outputs")
        df_query= get_query(data)
        df_query['sql']=df_query['sql'].apply(lambda x: get_strings(x))
        #sql제거
        if 'qeury' in df.columns:
            df['query']= df['query'].apply(lambda x: 'sql있음' if len(str(x))>10 else  '')
        
        wrt_file=json_file+'.xlsx'
        with pd.ExcelWriter(wrt_file) as writer:  
              df.to_excel(writer, sheet_name='json',index=True),
              df_inFile.to_excel(writer, sheet_name='input',index=False),
              df_outFile.to_excel(writer, sheet_name='output',index=False),
              df_query.to_excel(writer, sheet_name='query',index=False)
        return None

    except:
        print('fail ',  '- ', json_file)
        logging.info(f'fail to process {json_file}')
        return 'fail ' +  '- '+ json_file

#-------------------------------------------------------------

logging.basicConfig(filename='jobs.log', level=logging.DEBUG)
logging.info('Started : %s'%time.strftime("%Y%m%d %H:%M:%S",time.localtime()))

control=pd.read_excel('jobs.xlsx',sheet_name='control',header=0)        
try:
    for seq, row in control.iterrows():
        if str(row['Action']).upper()=='Y':
            result = fileCheck(row)
            if result[0]!=None:
                raise Exception(result)
                
            json_file=result[1]
            data=result[2]
            res = procData(json_file,data)
            if res!=None:
                raise Exception(res)     
            print("succ - ",json_file)
            
except  Exception as e: 
        print(f'error occured..{str(e)}')
        logging.info('%s ← error occured at..%s' %(str(e),time.strftime("%Y%m%d %H:%M:%S",time.localtime())))
        pass

##끝  아래는 디버깅용

In [ ]:
## 디버깅을 위한 로직

import json
import pandas as pd
# 파일에 대한 컬럼정보 추출
def get_columns(blocks,fType):
    cols=['block_id','raw_file_nm','col_id','nm','desc','calc_expression']
    # 각 블럭내 inputs정보 
    df_file = pd.DataFrame(columns=cols)
    for block in data['blocks']:
        if len(block[fType])!=0:
            for temp in block[fType]:
                for inp in temp:
                    if str(type(inp))=="<class 'dict'>":
                        for col in inp['ffd']['columns']:
                            lst= [block['block_id'],del_path(inp['raw_file_nm']),col['col_id'],col['nm'],col['desc'],col['calc_expression']]
                            dd=pd.DataFrame(lst).T
                            dd.columns=cols
                            df_file=df_file.append(dd)

    return df_file

def get_columns2(blocks,fType):
    cols=['block_id','raw_file_nm','col_id','nm','desc','calc_expression']
    # 각 블럭내 inputs정보 
    df_file = pd.DataFrame(columns=cols)
    for block in data['blocks']:
        if len(block[fType])!=0:
            for inp in block[fType]:
                if (str(type(inp))=="<class 'dict'>") & ('ffd' in inp.keys()):
                    for col in inp['ffd']['columns']:
                        lst= [block['block_id'],del_path(inp['raw_file_nm']),col['col_id'],col['nm'],col['desc'],col['calc_expression']]
                        dd=pd.DataFrame(lst).T
                        dd.columns=cols
                        df_file=df_file.append(dd)
    return df_file

json_path='D:/kpayins/02.참조모델/03 배치/KKO_tera_20200708/'
# json_path='k:/7000. 팀별공유자료/7200. 업무개발/7220. 자동차보험/04.배치\KKO_tera_20200708/KKO_tera_20200708/ccm/'
# json_file='TSP-btCcrM862_8607.json'
json_file='TSP-btCcrJ301_7807.json'
# json_file='TSP-btCcrM420_7582.json'

json_data=open(json_path+json_file, encoding='UTF8')
data = json.load(json_data)

df = pd.DataFrame()
# for block in data['blocks']:
#     df=df.append(block,ignore_index=True)
df=pd.json_normalize(data['blocks'])

df['input_cnt']=df.inputs.apply(lambda x: len(x))
df['output_cnt']=df.outputs.apply(lambda x: len(x))

df['input_files']=df.inputs.apply(lambda x: ex(x,'raw_file_nm'))
df['output_files']=df.outputs.apply(lambda x: ex(x,'raw_file_nm'))
df['input_files']=df['input_files'].apply(lambda x: del_path(x))
df['output_files']=df['output_files'].apply(lambda x: del_path(x))
if 'outfile1' in df.columns: 
    df['outfile1']=df['outfile1'].apply(lambda x: del_path(str(x))).replace('nan',"")
# df=df.applymap(lambda x: str(x).replace("$$[BAT_DATA_HOME]/ccr/","").replace("'",""))
df.drop(columns=['inputs','outputs'],inplace=True)

#in,out col추출
df_inFile=h=get_columns2(data,"inputs")
df_outFile=h=get_columns2(data,"outputs")
df_query= get_query(data)
df_query['sql']=df_query['sql'].apply(lambda x: get_strings(x))

if 'qeury' in df.columns:
    df['query']= df['query'].apply(lambda x: 'sql있음' if len(str(x))>10 else  '')

wrt_path='d:/kpayins/02.참조모델/03 배치/'
wrt_file=json_file+'.xlsx'
with pd.ExcelWriter(wrt_path+wrt_file) as writer:  
      df.to_excel(writer, sheet_name='json',index=True),
      df_inFile.to_excel(writer, sheet_name='input',index=False),
      df_outFile.to_excel(writer, sheet_name='output',index=False),
      df_query.to_excel(writer, sheet_name='query',index=False)  

print("end")

In [ ]:
# # len(data['blocks'][2]['outputs'])
inp=data['blocks'][2]
print(type(inp))
# if str(type(inp))=="<class 'dict'>":
#      for col in inp['ffd']['columns']:
#          lst= [block['block_id'],del_path(inp['raw_file_nm']),col['col_id'],col['nm'],col['desc'],col['calc_expression']]

# # temp=data['blocks'][2]['inputs']
# # for inp in temp:
#     print(inp['ffd'])
if 'cmd' in inp.keys():
    print(1)

In [ ]:
cols=['block_id','raw_file_nm','col_id','nm','desc','calc_expression']
    # 각 블럭내 inputs정보 
fType='inputs'
df_file = pd.DataFrame(columns=cols)
for block in data['blocks']:
        if len(block[fType])!=0:
            print(type(block[fType]))
            for temp in block[fType]:
                for inp in temp:
                    if str(type(inp))=="<class 'dict'>":
                        for col in inp['ffd']['columns']:
                            lst= [block['block_id'],del_path(inp['raw_file_nm']),col['col_id'],col['nm'],col['desc'],col['calc_expression']]
                            dd=pd.DataFrame(lst).T
                            dd.columns=cols
                            df_file=df_file.append(dd)
#                     else: print(inp)


In [ ]:
del get_columns

In [ ]:
import os
for root,dirs,files in os.walk('d:/'):
    for fname in files:
        full_name=os.path.join(root,fname)
        print(full_name)

In [ ]:
def cols_proc(col):
    if col['calc_expression']==None:
        col['calc_expression']=""
    return col['col_id'] +','+col['nm'] +','+ col['desc'] +','+ col['calc_expression']


## json구조
json -> {[blocks],desc11,grp_nm,name} -> [{inputs}] -> 
  {cond,ffd{columns,desc,ffd_id,nm},ffd_id,raw_file_nm,role_type_cd,sequence} 

data['blocks'][0]['inputs'][0]

In [ ]:
#dict구조 컬럼을 나눔, inputs,outputs
df1 = pd.concat((pd.DataFrame([x for x in df.inputs])),
        (pd.DataFrame([x for x in df.outputs]))], axis=1, keys=['blk','inp','oup'])

In [ ]:
df1=pd.DataFrame()
for d in df.inputs: #리스트
    for x in d:   #dict:cond,ffd,ffd_id,raw_file_nm,role_type_cd,sequence
                  ## dict->ffd  
        #         print(pd.DataFrame(x))
        dt=pd.DataFrame(x)
        print(dt.columns)
#         df1.append(dt)

In [ ]:
#모든 컬럼이 dict구조인 경우
dfs = [pd.DataFrame([x for x in df[col]]) for col in df.columns]
df1 = pd.concat(dfs, axis=1, keys=df.columns)
with pd.ExcelWriter('d:/kpayins/json.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='json',index=True)

In [ ]:
for block in data['blocks']:
#     print('------------------------')
    print(block['block_id'],block['depend_on'],block['nm'])
    if len(block['inputs'])>1:
        print('inp = ', len(block['inputs']))
        for inp in block['inputs']:
            for k,v in inp.items():
                if len(v)>0:
                    if k=='raw_file_nm':
                        print(v) 
    if len(block['outputs'])>1:
        print('out = ', len(block['outputs']))
        for out in block['outputs']:
            for k,v in out.items():
                if len(v)>0:
                    if k=='raw_file_nm':
                        print(v) 


In [ ]:
keyLi=['block_id','cmd', 'depend_on', 'inputs', 'join_cond', 'join_type', 'key_map', 'nm', 'outputs', 'type']
outLi=['block_id','depend_on','nm','type', 'key_map','input_cnt','output_cnt','input_files','output_files','inputs','outputs']
